In [231]:
import requests # type: ignore
import os
from bs4 import BeautifulSoup # type: ignore
from dotenv import load_dotenv # type: ignore
from requests.exceptions import RequestException # type: ignore
from fake_useragent import UserAgent # type: ignore
from tenacity import (retry,retry_if_exception_type,stop_after_attempt , wait_exponential)
from datetime import date ,time

In [232]:
load_dotenv()

True

In [233]:
base_url =  os.getenv("BASE_URL")
matches_url = os.getenv("MATCHES_URL")

In [234]:
def genrate_random_useragent():
    # Create a UserAgent object
    ua = UserAgent()
    # Get a random user agent string
    random_user_agent = ua.random
    return random_user_agent



In [235]:
@retry(
    reraise = True,
    stop = stop_after_attempt(5),
    wait = wait_exponential(multiplier = 1 , min = 1 , max = 10),
    retry = (retry_if_exception_type(RequestException))
)
def requests_with_retry(url):
    
    user_agent = genrate_random_useragent()
    headers = {'User-Agent': user_agent}
    
    try:
        res = requests.get(url=url,headers=headers)
        res.raise_for_status()
        print(res.request.headers)
        return res
    except RequestException as e:
        print(f"Exception occured : {e}")
        raise RequestException
           
    

In [292]:
response = requests_with_retry(url=matches_url)
soup = BeautifulSoup(response.content)


Exception occured : 403 Client Error: Forbidden for url: https://www.hltv.org/matches
Exception occured : 403 Client Error: Forbidden for url: https://www.hltv.org/matches
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [293]:
upcomming_matches_sections = soup.find_all('div',class_='upcomingMatchesSection')

In [ ]:
match = {
    'matchDate' : date,
    'matchTime' : time,
    'matchTeam1': str,
    'matchTeam2': str,
    'matchEventName': str,
    'matchMeta':str,
    'matchRating':int,
}

In [306]:
section = upcomming_matches_sections[0]

for section in upcomming_matches_sections:
    print(section.find('span',class_ ='matchDayHeadline'))

<span class="matchDayHeadline">Wednesday - 2024-05-29</span>
<span class="matchDayHeadline">Thursday - 2024-05-30</span>
<span class="matchDayHeadline">Friday - 2024-05-31</span>
<span class="matchDayHeadline">Saturday - 2024-06-01</span>
<span class="matchDayHeadline">Sunday - 2024-06-02</span>
<span class="matchDayHeadline">Monday - 2024-06-03</span>
<span class="matchDayHeadline">Tuesday - 2024-06-04</span>
<span class="matchDayHeadline">Wednesday - 2024-06-05</span>
<span class="matchDayHeadline">Thursday - 2024-06-06</span>
<span class="matchDayHeadline">Friday - 2024-06-07</span>
<span class="matchDayHeadline">Saturday - 2024-06-08</span>
<span class="matchDayHeadline">Sunday - 2024-06-09</span>
<span class="matchDayHeadline">Monday - 2024-06-10</span>
<span class="matchDayHeadline">Tuesday - 2024-06-11</span>
<span class="matchDayHeadline">Wednesday - 2024-06-12</span>
<span class="matchDayHeadline">Thursday - 2024-06-13</span>


In [315]:

matches = []
match_dictionary = {'matchDate' : date,
    'matchTime' : time,
    'matchTeam1': str,
    'matchTeam2': str,
    'matchEventName': str,
    'matchMeta':str,
    'matchRating':int,}
for section in upcomming_matches_sections:
    match_date_tag = section.find('span',class_ = "matchDayHeadline")
    match_date_tag.text
    match_dictionary['matchDate'] = match_date_tag.text
    
    upcoming_matches = section.find_all('div', class_ = 'upcomingMatch')
    print((upcoming_matches))
    # for match in upcoming_matches:
    #     # match_event_name_tag = match.find_all('div', class_ = 'matchEventName')
        
    #     match_meta_tag = match.find('div',class_= 'matchMeta')
    #     match_rating = match['stars']
        
    #     match_dictionary['matchEventName'] = 'ABHI LITE LEE RAHE HAI'
    #     match_dictionary['matchMeta'] = match_meta_tag.text
    #     match_dictionary['matchRating'] = match_rating
        
        
        # if match.find_all('div', class_ = 'matchInfoEmpty'):
        #     match_dictionary['matchTeam1'] = 'TBD'
        #     match_dictionary['matchTeam2'] = 'TBD'
        # else:
        #     teams = match.find_all('div' , class_ = 'matchTeamName')
        #     if len(teams) == 1:
        #         match_dictionary['matchTeam2'] = 'TBD'
        #         match_dictionary['matchTeam1'] = teams[0].text
        #     else:
        #         match_dictionary['matchTeam1'] = teams[0].text
        #         match_dictionary['matchTeam2'] = teams[1].text
                
        # matches.append(match_dictionary)            
        
                
            
            
            


15
26
24
31
19
4
13
20
17
15
10
9
3
2
2
2


In [310]:
matches

[{'matchDate': 'Thursday - 2024-06-13',
  'matchTime': datetime.time,
  'matchTeam1': str,
  'matchTeam2': str,
  'matchEventName': 'ABHI LITE LEE RAHE HAI',
  'matchMeta': 'bo5',
  'matchRating': '0'},
 {'matchDate': 'Thursday - 2024-06-13',
  'matchTime': datetime.time,
  'matchTeam1': str,
  'matchTeam2': str,
  'matchEventName': 'ABHI LITE LEE RAHE HAI',
  'matchMeta': 'bo5',
  'matchRating': '0'},
 {'matchDate': 'Thursday - 2024-06-13',
  'matchTime': datetime.time,
  'matchTeam1': str,
  'matchTeam2': str,
  'matchEventName': 'ABHI LITE LEE RAHE HAI',
  'matchMeta': 'bo5',
  'matchRating': '0'},
 {'matchDate': 'Thursday - 2024-06-13',
  'matchTime': datetime.time,
  'matchTeam1': str,
  'matchTeam2': str,
  'matchEventName': 'ABHI LITE LEE RAHE HAI',
  'matchMeta': 'bo5',
  'matchRating': '0'},
 {'matchDate': 'Thursday - 2024-06-13',
  'matchTime': datetime.time,
  'matchTeam1': str,
  'matchTeam2': str,
  'matchEventName': 'ABHI LITE LEE RAHE HAI',
  'matchMeta': 'bo5',
  'match